<a href="https://colab.research.google.com/github/fishenzone/Projects/blob/main/projects/test_tasks/Gorod/books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import warnings
warnings.filterwarnings('ignore')
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

# Тестовое задание: 

По данным подтвердить или опровергнуть гипотезу об уменьшении чека в отчетном периоде (апрель 2019), в сравнении с предыдущим месяцем (март 2019). 
Сделать выводы. 

sample 1 - оформленные заказы
sample 2 - состав оформленных заказов 

Date_order - дата заказа
Order_id - номер заказа 
User_id - клиент
Price - стоимость заказа 
Price_delivery - стоимость доставки
Bonus - оплата баллами
Discount_price - скидка на товар 
Product_id - товар 
Quantity - количество товара 

In [ ]:
def read(url):
  url='https://drive.google.com/uc?id=' + url.split('/')[-2]
  df = pd.read_csv(url, sep=';', encoding='latin1', index_col=0)
  return df

In [ ]:
sample1 = read('https://drive.google.com/file/d/10RI4uq3lQbygjsAIp3vDCBOBx-lZpY7h/view?usp=share_link')
sample2 = read('https://drive.google.com/file/d/1UyMbXMFCsPTQFfH2WybrILa2fLpPypxD/view?usp=share_link')
sample1.rename(columns={sample1.columns[0]: 'date_order'}, inplace=True)

In [ ]:
def convert(df, col1, col2):
  mask = pd.to_numeric(df[col1], errors='coerce').isna() | pd.to_numeric(df[col2], errors='coerce').isna()
  df = df[~mask]
  df[[col1, col2]] = df[[col1, col2]].astype('float64')
  return df

In [ ]:
merged_df = pd.merge(sample1, sample2, on='order_id')
merged_df['order_month'] = pd.to_datetime(merged_df['date_order'], format='%d.%m.%Y').dt.month

filtered_df = merged_df[(merged_df['order_month'] == 3) | (merged_df['order_month'] == 4)]
filtered_df = convert(filtered_df, 'bonus', 'discount_price')
filtered_df['total_order_value'] = filtered_df['price_x'] + filtered_df['price_delivery'] - filtered_df['bonus'] - filtered_df['discount_price']

march_avg = filtered_df[filtered_df['order_month'] == 3]['total_order_value'].mean()
april_avg = filtered_df[filtered_df['order_month'] == 4]['total_order_value'].mean()

pvalue = ttest_ind(filtered_df[filtered_df['order_month'] == 3]['total_order_value'], filtered_df[filtered_df['order_month'] == 4]['total_order_value']).pvalue

if pvalue < 0.05:
    print(f"The average order value in April 2019 is significantly different from March 2019 (p-value = {pvalue:.3f})")
else:
    print(f"The average order value in April 2019 is not significantly different from March 2019 (p-value = {pvalue:.3f})")


The average order value in April 2019 is significantly different from March 2019 (p-value = 0.000)
